In [ ]:
from flask import Flask,jsonify,render_template
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [ ]:
app = Flask(__name__)

restaurant_data = json.loads("../resataurant_data.json")
restaurnt_df = pd.DataFrame(restaurant_data)

#--SQL STUFF
# engine = create_engine("sqlite:///Resources/votes.sqlite")
# base = automap_base()
# base.prepare(autoload_with=engine)
#-- 

@app.route("/")
def home():
    print("YOLO CHOLO")
    # Start at the homepage.
    # List all the available routes.
    return (
        f"Welcome to the Portland Eats API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/voting<br/>"
        f"/api/v1.0/map/<br/>"
        f"/api/v1.0/chart/<br/>"
        )



@app.route("/api/v1.0/voting2")
def nothome2():
    # Convert the query results from your precipitation analysis (i.e. retrieve only the last 12 months of data) to a dictionary using date as the key and prcp as the value.
    # last_year_prcp =last_year_to_obj()
    # Return the JSON representation of your dictionary.
    return render_template('templates/voting.html')

@app.route("/api/v1.0/map")
def somthing():
    pass
    # Return a JSON list of stations from the dataset.
    # return jsonify(list(ly_df["station"].unique()))

@app.route("/api/v1.0/chart")
def waitasecond():
    # Query the dates and temperature observations of the most-active station for the previous year of data.
    pass
    # Return a JSON list of temperature observations for the previous year.
    # return busy_df.set_index("date")["tobs"].to_dict()


if(__name__=="__main__"):
    app.run(debug=True)